Plot the summary of statistics (energy balance, optimal capacity, ...) 

**Prerequisites**

* Execute "make_stats_dict.py" before using this notebook. The script make_stats_dict creates csv for the metrics for all runs of multiple config files. 
* Install the environment envs/pypsa-earth-vis.yaml

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd
idx_slice = pd.IndexSlice
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Comment out for debugging and development
warnings.simplefilter(action='ignore', category=DeprecationWarning) # Comment out for debugging and development
import plotly.express as px

from plot_helpers import (
    chdir_to_parent_dir,
    read_stats_dict,
    drop_index_levels, 
    update_layout, 
    prepare_dataframe,
    get_supply_demand_from_balance,
    rename_electricity, 
    rename_h2,
    rename_gas,
    rename_oil,
    rename_co2,
    rename_costs,
    rename_to_upper_case,
    colors,
    nice_title,
    save_plotly_fig)

chdir_to_parent_dir()

In [ ]:
run_name_prefix = "H2G_A" # Experiment name

sdir = Path.cwd() / "results"/ f"{run_name_prefix}_summary_20250523"
sdir.mkdir(exist_ok=True, parents=True)


In [ ]:
balance_dict = read_stats_dict("balance_dict", sdir, keys=["AC", "H2", "oil", "gas", "co2 stored", "co2"])
optimal_capacity_dict = read_stats_dict("optimal_capacity_dict", sdir, keys=["AC", "H2"])
costs_dict = read_stats_dict("costs_dict", sdir, keys=["capex", "opex"])

# simplify index for plotting: define a scen_str column and drop all other columns
#NB: the following is specific to the experiment

index_levels_to_drop=["simpl", "clusters", "ll", "opts", "sopts", "discountrate", "demand"]

for key in balance_dict.keys():
    balance_dict[key] = drop_index_levels(balance_dict[key], to_drop=index_levels_to_drop)

for key in optimal_capacity_dict.keys():
    optimal_capacity_dict[key] = drop_index_levels(optimal_capacity_dict[key], to_drop=index_levels_to_drop)
    
for key in costs_dict.keys():
    costs_dict[key] = drop_index_levels(costs_dict[key], to_drop=index_levels_to_drop)

# Plotting

In [ ]:
print(balance_dict["AC"].index.names)
print(balance_dict["AC"].index)

In [ ]:
# select and define index group (multiindex) for plots: which solutions to show at once

idx_group = idx_slice[["H2G_A"],["ZA"],[2050],:]
idx_group_name = ""

print(idx_group_name) 
print(idx_group)

In [ ]:
width = 700
heights = {"1": 300,"2": 600,"3": 700,}

fig_kwargs = dict(
    x="scen", 
    y="value", 
    color="variable", 
    facet_col="year", 
    facet_row="country",
    barmode="relative", 
    text="value", 
    #facet_col_wrap=1, facet_col_spacing=0.07,
    width=width, 
    height=heights["2"],
    text_auto =".0f",
)

scen_order = [] # option to specify order of scenarios in the plot



In [ ]:
# balance_dict["gas"].query("year == 2050 and country == 'ZA'").T

In [ ]:
# balance_dict["co2 stored"].query("year == 2050 and country == 'ZA'").T

## Energy market balance | Electricity

In [ ]:
df = prepare_dataframe(balance_dict["AC"], idx_group)
df.variable = df.variable.map(rename_electricity).map(rename_to_upper_case)

(supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df)

In [ ]:
fig_name = "electricity_supply_TWh"
fig = px.bar(
    supply_df, 
    **fig_kwargs,
    labels={"value":"Electricity supply in TWh<sub>el</sub>",
            "year":"",
            "variable":"", 
            "scen":""
            },
    color_discrete_map=colors["electricity"],
    category_orders={
        "variable":list(colors["electricity"].keys()),
        "scen": scen_order
        },
    #facet_col_wrap=1, facet_col_spacing=0.07,
)
fig.update_layout(legend_traceorder="reversed",)
update_layout(fig)

print("Totals per scenario and year:", supply_sum_df)
fig.show()
#save_plotly_fig(supply_df, fig, sdir, f"{idx_group_name}_{fig_name}")

In [ ]:
fig_name = "electricity_demand_TWh"
fig = px.bar(
    demand_df, 
    **fig_kwargs,
    color_discrete_map=colors["electricity"],
    category_orders={
        "variable":list(colors["electricity"].keys()),
        "scen": scen_order
        },
    labels={"value":"Electricity demand in TWh<sub>el</sub>","year":"",
            "variable":"", 
            "scen":""},
)
fig.update_layout(legend_traceorder="reversed",)
update_layout(fig)

print("Totals per scenario and year:", demand_sum_df)
fig.show()
#save_plotly_fig(demand_df, fig, sdir, f"{idx_group_name}_{fig_name}")


## Energy and feedstock market balance | Hydrogen

In [ ]:
df = prepare_dataframe(balance_dict["H2"], idx_group)
df = df[df["value"]!=0]
df.variable = df.variable.map(rename_h2)

(supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df)

In [ ]:
fig_name = "hydrogen_balance_TWh"
fig = px.bar(
    df, 
    **fig_kwargs,
    color_discrete_map=colors["hydrogen"],
    category_orders={
        "variable":list(colors["hydrogen"].keys()),
        "scen": scen_order
        },
    #facet_col_spacing=0.1,
    labels={"value":"H2 balance in TWh","year":"",
            "variable":"", 
            "scen": ""},

)
update_layout(fig)

print("Totals per scenario and year:", demand_sum_df)
fig.show()
#save_plotly_fig(df, fig, fig, sdir, f"{idx_group_name}_{fig_name}")


## Energy and feedstock market balance | Liquid fuel

In [ ]:
df = prepare_dataframe(balance_dict["oil"], idx_group)
df = df[df["value"]!=0]
df.variable = df.variable.map(rename_oil)

(supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df)

In [ ]:
fig_name = "liquid_fuel_balance_TWh"
fig = px.bar(
    df, 
    **fig_kwargs,
    color_discrete_map=colors["oil"],
    category_orders={
        "variable":list(colors["oil"].keys()),
        "scen": scen_order
        },
    #facet_col_spacing=0.1,
    labels={"value":"Liquid fuel balance in TWh","year":"",
            "variable":"", 
            "scen":""},

)
update_layout(fig)

print("Totals per scenario and year:", demand_sum_df)
fig.show()
#save_plotly_fig(df, fig, fig, sdir, f"{idx_group_name}_{fig_name}")

## Energy and feedstock market balance | CH4

In [ ]:
df = prepare_dataframe(balance_dict["gas"], idx_group)
df = df[df["value"]!=0]
df.variable = df.variable.map(rename_gas)

(supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df)

In [ ]:
fig_name = "CH4_balance_TWh"
fig = px.bar(
    df, 
    **fig_kwargs,
    color_discrete_map=colors["gas"],
    category_orders={
        "variable":list(colors["gas"].keys()),
        "scen": scen_order
        },
    #facet_col_spacing=0.1,
    labels={"value":"CH4 balance in TWh","year":"",
            "variable":"", 
            "scen":""},

)
update_layout(fig)

print("Totals per scenario and year:", demand_sum_df)
fig.show()
#save_plotly_fig(df, fig, fig, sdir, f"{idx_group_name}_{fig_name}")

## Energy and feedstock market balance | CO2

In [ ]:
df = prepare_dataframe(balance_dict["co2 stored"], idx_group)
df = df[df["value"]!=0]

df.variable = df.variable.map(rename_co2)

(supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df)

In [ ]:
fig = px.bar(
    df, 
    **fig_kwargs,
    color_discrete_map=colors["co2"],
    category_orders={
        "variable":list(colors["gas"].keys()),
        "scen": scen_order
        },
    #facet_col_spacing=0.1,
    labels={"value":"CO<sub>2</sub> Capture (+) and Usage (-) in Mt","year":"","variable":"", "scen":""},
)
update_layout(fig)

print("Totals per scenario and year:", demand_sum_df)
fig.show()
#save_plotly_fig(df, fig, fig, sdir, f"{idx_group_name}_CO2_capture_and_usage_Mt")

## CO2 atmosphere balance

In [ ]:
df = prepare_dataframe(balance_dict["co2"], idx_group)

df.variable = df.variable.map(rename_co2)

(supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df)

In [ ]:
fig_name = "co2_emissions_mt"
fig = px.bar(
    df, 
    **fig_kwargs,
    labels={"value":"CO2 emissions in Mt<sub>CO2</sub>",
            "year":"",
            "variable":"", 
            "scen":""
            },
    color_discrete_map=colors["co2"],
    category_orders={
        "variable":list(colors["co2"].keys()),
        "scen": scen_order
        },
    #facet_col_wrap=1, facet_col_spacing=0.07,
)
fig.update_layout(legend_traceorder="reversed",)
update_layout(fig)

print("Totals per scenario and year:", supply_sum_df)
fig.show()
#save_plotly_fig(supply_df, fig, sdir, f"{idx_group_name}_{fig_name}")

In [ ]:
df = prepare_dataframe(optimal_capacity_dict["AC"], idx_group)
df = df[df["value"]<1e6] #drop loadshedding capacity
df.variable = df.variable.map(rename_electricity).map(rename_to_upper_case)

(supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df)

In [ ]:
fig_name = "electricity_installed_capacity_GW"
fig = px.bar(
    df, 
    **fig_kwargs,
    #barmode="stack",
    color_discrete_map=colors["electricity"],
    category_orders={
        "variable":list(colors["electricity"].keys()),
        "scen": scen_order
        },
    #facet_col_spacing=0.1,
    #range_y=(0,supply_sum_df.max().value*1.1),
    labels={"value":"Installed capacity in GW<sub>el</sub>","year":"",
            #"variable":"",
            "scen":""},
)
update_layout(fig)

print("Totals per scenario and year:", supply_sum_df)
fig.show()
#save_plotly_fig(df, fig, fig, sdir, f"{idx_group_name}_{fig_name}")

In [ ]:
df = prepare_dataframe(optimal_capacity_dict["H2"], idx_group)
df = df[df["value"]<1e6] #drop loadshedding capacity
df.variable = df.variable.map(rename_h2)

(supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df)

In [ ]:
fig_name = "hydrogen_installed_capacity_GW"
fig = px.bar(
    df, 
    **fig_kwargs,
    color_discrete_map={"H2 Electrolysis":"#179c7d"},
    category_orders={
        #"variable":list(colors["gas"].keys())
        "scen": scen_order
        },
    labels={"value":"Installed capacity in GW<sub>H2</sub>","year":"","variable":"","scen":""},
)
update_layout(fig)

print("Totals per scenario and year:", supply_sum_df)
fig.show()
#save_plotly_fig(df, fig, fig, sdir, f"{idx_group_name}_{fig_name}")

In [ ]:
stats_df = pd.concat([costs_dict["capex"],costs_dict["opex"]],axis=0)
df = prepare_dataframe(stats_df, idx_group)
df.variable = df.variable.map(rename_costs) # ToDo rename important cost components
df = df.groupby(["run_name_prefix", "scen", "year", "country", "variable"], as_index=False).sum().round(1)
df = df[df["value"]!=0]

# (supply_df, supply_sum_df, demand_df, demand_sum_df) = get_supply_demand_from_balance(df, threshold=0., round=1)

In [ ]:
fig_name = "system_costs_billion_eur" # expenditure and export revenue
fig_kwargs["height"] = heights["3"]
fig_kwargs["text_auto"] = ".1f"
fig = px.bar(
    df, 
    **fig_kwargs,
    color_discrete_map=colors["costs"],
    category_orders={
        "variable":list(colors["costs"].keys()),
        #"scen": scen_order
        },
    facet_col_spacing=0.1,
    labels={"value":"System costs in billion EUR20XX/year","year":"", 
            #"variable":"", 
            "scen":""},
)
update_layout(fig)

print("Totals per scenario and year:", supply_sum_df)
fig.show()
#save_plotly_fig(df, fig, fig, sdir, f"{idx_group_name}_installed_capacity_AC_GW")

## Relative costs

costs / total energy consumption

## Water consumption